# Exploring the Structure of the Data

### Import

In [1]:
pwd()

u'/home/ubuntu/twitterMovies-vader/sandbox'

In [2]:
import os
os.chdir(r'/home/ubuntu/twitterMovies-vader')

#pwd()

In [3]:
#alluxio://master001:19998/

# alluxio://master001:19998/DeerAntMan/*.gz
# /twitter/movie/DeerAntMan/*.gz

df = spark.read.json('alluxio://master001:19998/twitter-chicago/DeerAntMan/*.gz')

In [4]:
df.printSchema()

root
 |-- actor: struct (nullable = true)
 |    |-- displayName: string (nullable = true)
 |    |-- favoritesCount: long (nullable = true)
 |    |-- followersCount: long (nullable = true)
 |    |-- friendsCount: long (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- image: string (nullable = true)
 |    |-- languages: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- link: string (nullable = true)
 |    |-- links: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- rel: string (nullable = true)
 |    |-- listedCount: long (nullable = true)
 |    |-- location: struct (nullable = true)
 |    |    |-- displayName: string (nullable = true)
 |    |    |-- objectType: string (nullable = true)
 |    |-- objectType: string (nullable = true)
 |    |-- postedTime: string (nullable = true)
 |    |-- preferredUsername: string (nullable = true)
 |    

### Getting the Number of rows

In [3]:
df.count()

192718

In [ ]:
df.columns

In [17]:
df.select('actor.displayName').first()

Row(displayName=u'Nicolas')

In [15]:
df.select('actor.preferredUsername').first()

Row(preferredUsername=u'NicoStarkiller')

In [ ]:
df.select('body').first()

In [ ]:
df.select('gnip').first()

In [ ]:
df.select('postedTime').first()

In [5]:
small_df = df.select('body', 'postedTime', 'retweetCount').na.drop()

In [6]:
small_df.count()

181507

## Formatting the date string using DateUtil (deprecated)

In [6]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

In [7]:
from dateutil.parser import parse as parse_date

In [22]:
def parsedate(dateString):
    try:
        return parse_date(dateString).date()
    except:
        pass

In [23]:
parsedate_udf = udf(parsedate)

In [24]:
small_df2 = small_df.withColumn('date', parsedate_udf(col('postedTime')))

In [25]:
small_df2.printSchema()

root
 |-- body: string (nullable = true)
 |-- postedTime: string (nullable = true)
 |-- retweetCount: long (nullable = true)
 |-- date: string (nullable = true)



In [26]:
small_df2.show(5)

+--------------------+--------------------+------------+--------------------+
|                body|          postedTime|retweetCount|                date|
+--------------------+--------------------+------------+--------------------+
|RT @Kotaku: The n...|2015-04-13T13:30:...|          11|java.util.Gregori...|
|RT @SuperheroFeed...|2015-04-13T13:30:...|         428|java.util.Gregori...|
|RT @SuperheroFeed...|2015-04-13T13:30:...|         426|java.util.Gregori...|
|Second Action-Pac...|2015-04-13T13:30:...|           0|java.util.Gregori...|
|RT @screenrant: '...|2015-04-13T13:30:...|           8|java.util.Gregori...|
+--------------------+--------------------+------------+--------------------+
only showing top 5 rows



ImportError: No module named spark.sql.types.DateType

## Formatting the Date String with PySpark inbuilt

In [7]:
small_df2 = small_df.withColumn('dateColumn', small_df['postedTime'].cast('date'))

In [32]:
small_df2.printSchema()

root
 |-- body: string (nullable = true)
 |-- postedTime: string (nullable = true)
 |-- retweetCount: long (nullable = true)
 |-- dateColumn: date (nullable = true)



In [8]:
small_df2.show(5)

+--------------------+--------------------+------------+----------+
|                body|          postedTime|retweetCount|dateColumn|
+--------------------+--------------------+------------+----------+
|RT @Kotaku: The n...|2015-04-13T13:30:...|          11|2015-04-13|
|RT @SuperheroFeed...|2015-04-13T13:30:...|         428|2015-04-13|
|RT @SuperheroFeed...|2015-04-13T13:30:...|         426|2015-04-13|
|Second Action-Pac...|2015-04-13T13:30:...|           0|2015-04-13|
|RT @screenrant: '...|2015-04-13T13:30:...|           8|2015-04-13|
+--------------------+--------------------+------------+----------+
only showing top 5 rows



In [11]:
small_df2.select('postedTime').first()

Row(postedTime=u'2015-04-13T13:30:53.000Z')

In [33]:
tweetsByDay = small_df2.groupby(small_df2.dateColumn).count()

In [37]:
tweetsByDay.describe().show()

+-------+------------------+
|summary|             count|
+-------+------------------+
|  count|               370|
|   mean|490.55945945945945|
| stddev|1122.6372130715913|
|    min|                 5|
|    max|             10942|
+-------+------------------+



## Can we get the date as a date and time variable

In [29]:
from pyspark.sql.functions import hour, minute, date_format

In [40]:
small_df2.select(hour('postedTime'), 
                 minute('postedTime'), 
                 date_format('postedTime', 'dd-MM-yyy HH:mm').alias('date_minute'),
                 'postedTime').show(1000)


+----------------+------------------+----------------+--------------------+
|hour(postedTime)|minute(postedTime)|     date_minute|          postedTime|
+----------------+------------------+----------------+--------------------+
|              15|                30|13-04-2015 15:30|2015-04-13T13:30:...|
|              15|                30|13-04-2015 15:30|2015-04-13T13:30:...|
|              15|                30|13-04-2015 15:30|2015-04-13T13:30:...|
|              15|                30|13-04-2015 15:30|2015-04-13T13:30:...|
|              15|                30|13-04-2015 15:30|2015-04-13T13:30:...|
|              15|                30|13-04-2015 15:30|2015-04-13T13:30:...|
|              15|                30|13-04-2015 15:30|2015-04-13T13:30:...|
|              15|                30|13-04-2015 15:30|2015-04-13T13:30:...|
|              15|                30|13-04-2015 15:30|2015-04-13T13:30:...|
|              15|                30|13-04-2015 15:30|2015-04-13T13:30:...|
|           

In [37]:
small_df2.select(date_format('postedTime', 'dd-MM-yyy HH:mm')).show(5)

+----------------------------------------+
|date_format(postedTime, dd-MM-yyy HH:mm)|
+----------------------------------------+
|                        13-04-2015 15:30|
|                        13-04-2015 15:30|
|                        13-04-2015 15:30|
|                        13-04-2015 15:30|
|                        13-04-2015 15:30|
+----------------------------------------+
only showing top 5 rows



In [28]:
spark.version


u'2.1.1'